In [1]:
import qsharp
from qsharp_widgets import SpaceChart, EstimateDetails, EstimatesOverview, EstimatesPanel

## Now we simulate quantum metrology experiment for (1) Shot-noise limit and (2) Heisenberg limit


### Hence we define necessary operations


In [2]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Math;

operation SetQubitState(desired : Result, target : Qubit) : Unit {
   if desired != M(target) {
       X(target);
   }
}


@EntryPoint()
operation Shotnoise(qubno : Int, ang : Double, count : Int) : Int[] {
    use qs = Qubit[qubno];
    mutable tcount = [0,size=qubno];
    for i in 0 .. qubno-1 {
        SetQubitState( Zero, qs[i]);
    }  
    for j in 0 ..count  {
        for i in 0 .. qubno-1 {
            H(qs[i]);
            R1(ang,qs[i]);
            H(qs[i]);
            let r= M(qs[i]);
            Reset(qs[i]);
            if r == One {
                if i==0 {
                set tcount = [tcount[i] + 1] + tcount[i + 1 .. Length(tcount)-1];
                }
                elif i==qubno-1{
                set tcount = tcount[0 .. i-1] + [tcount[i] + 1];
                }
                else {
                set tcount = tcount[0 .. i-1] + [tcount[i] + 1] + tcount[i + 1 .. Length(tcount)-1];
                }
            }
        }
    }
    return tcount;
}
operation Heisenberg(qubno : Int, ang : Double, count : Int) : Int {
    use qs = Qubit[qubno];
    mutable tcount = 0;
    for i in 0 .. qubno-1 {
        SetQubitState( Zero, qs[i]);
    } 
    for j in 0 .. count {
        H(qs[0]);
        for i in 1 .. qubno-1 {
            CNOT(qs[0],qs[i])
        }
        for i in 0 .. qubno-1 {
            R1(ang,qs[i]);
        }
        for i in 1 .. qubno-1 {
            CNOT(qs[0],qs[i])
        }
        H(qs[0]);
        let r= M(qs[0]);
        for i in 0 .. qubno-1 {
            Reset(qs[i]);
        }
        if r == One {
            set tcount+=1;
        }
    }
    return tcount;
}


###

In [20]:
import numpy as np
no_of_qubits=25
angle=1.0
delta=0.001
no_shots=100000
SN1 = qsharp.run("Shotnoise("+str(no_of_qubits)+","+str(angle)+","+str(no_shots)+")", shots=5)
SN2 = qsharp.run("Shotnoise("+str(no_of_qubits)+","+str(angle+delta)+","+str(no_shots)+")", shots=5)

In [21]:
HL1 = qsharp.run("Heisenberg("+str(no_of_qubits)+","+str(angle)+","+str(no_shots)+")", shots=5)
HL2 = qsharp.run("Heisenberg("+str(no_of_qubits)+","+str(angle+delta)+","+str(no_shots)+")", shots=5)

In [22]:
#print(np.mean(SN1,axis=0))
#print(no_of_qubits*no_shots-2*np.sum(np.mean(SN1,axis=0)))
#print(no_of_qubits*no_shots-2*np.sum(np.mean(SN2,axis=0)))
#print(np.mean(HL1))
#print(no_shots-2*np.mean(HL1))
#print(no_shots-2*np.mean(HL2))

In [23]:
Msn_avg=np.sqrt(4*np.sum(np.mean(SN1,axis=0))*no_shots-4*np.sum(np.mean(SN1,axis=0)**2))
dMsn_avg=(abs(2*np.sum(np.mean(SN2,axis=0))-2*np.sum(np.mean(SN1,axis=0))))/delta
print("The shot-noise sensitivity limit in quantum metrology with "+str(no_of_qubits)+" qubits is found to be :",Msn_avg/dMsn_avg)

The shot-noise sensitivity limit in quantum metrology with 25 qubits is found to be : 0.20332606850599153


In [24]:
Mhl_avg=np.sqrt(4*no_shots*np.mean(HL1)-4*np.mean(HL1)**2)
dMhl_avg=(abs(2*np.mean(HL1)-2*np.mean(HL2)))/delta
print("The Heisenberg sensitivity limit in quantum metrology with "+str(no_of_qubits)+" qubits is found to be :",Mhl_avg/dMhl_avg)

The Heisenberg sensitivity limit in quantum metrology with 25 qubits is found to be : 0.0446689112984856


In [25]:
result = qsharp.estimate(
    "Shotnoise("+str(no_of_qubits)+","+str(angle)+","+str(no_shots)+")",
    [
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" }},
        {
        "qubitParams": { "name": "qubit_gate_ns_e4" }},
        {
        "qubitParams": { "name": "qubit_gate_us_e3" }},
        {
        "qubitParams": { "name": "qubit_gate_us_e4" }},
        {
        "qubitParams": { "name": "qubit_maj_ns_e4" },
        "qecScheme": { "name": "floquet_code" }},
        {
        "qubitParams": { "name": "qubit_maj_ns_e6" },
        "qecScheme": { "name": "floquet_code" }},
    ]
)
result[:]

Item,0,1,2,3,4,5
Runtime,1 mins,32 secs,1 days,13 hours,24 secs,11 secs
rQOPS,7.86M,15.00M,5.24k,10.00k,20.00M,44.00M
Physical qubits,4.09M,770.53k,1.88M,514.49k,7.38M,312.31k
Item,0,1,2,3,4,5
Logical algorithmic qubits,66,66,66,66,66,66
Algorithmic depth,7.30M,7.30M,7.30M,7.30M,7.30M,7.30M
Logical depth,7.30M,7.30M,7.30M,7.30M,7.30M,7.30M
Clock frequency,119.05k,227.27k,79,152,303.03k,666.67k
Number of T states,57.50M,57.50M,57.50M,57.50M,57.50M,57.50M
Number of T factories,121,131,103,103,139,115


In [26]:
result = qsharp.estimate(
    "Heisenberg("+str(no_of_qubits)+","+str(angle)+","+str(no_shots)+")",
    [
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" }},
        {
        "qubitParams": { "name": "qubit_gate_ns_e4" }},
        {
        "qubitParams": { "name": "qubit_gate_us_e3" }},
        {
        "qubitParams": { "name": "qubit_gate_us_e4" }},
        {
        "qubitParams": { "name": "qubit_maj_ns_e4" },
        "qecScheme": { "name": "floquet_code" }},
        {
        "qubitParams": { "name": "qubit_maj_ns_e6" },
        "qecScheme": { "name": "floquet_code" }},
    ]
)
result[:]

Item,0,1,2,3,4,5
Runtime,41 secs,22 secs,17 hours,9 hours,16 secs,7 secs
rQOPS,7.86M,15.00M,5.24k,10.00k,20.00M,44.00M
Physical qubits,6.06M,1.14M,2.76M,756.49k,10.97M,462.79k
Item,0,1,2,3,4,5
Logical algorithmic qubits,66,66,66,66,66,66
Algorithmic depth,4.90M,4.90M,4.90M,4.90M,4.90M,4.90M
Logical depth,4.90M,4.90M,4.90M,4.90M,4.90M,4.90M
Clock frequency,119.05k,227.27k,79,152,303.03k,666.67k
Number of T states,57.50M,57.50M,57.50M,57.50M,57.50M,57.50M
Number of T factories,180,195,153,153,207,172
